In [1]:
%pip install gensim
%pip install tensorflow

In [2]:
# DT/RF viz packages (explainability)
!pip install pydotplus
!pip install graphviz
# ML packages
!pip install tensorflow
!pip install keras
!pip install h5py

import gensim
import glob
import imageio
import itertools
import math
import nltk
import operator
import os
import pickle
import pydot
import pydotplus
import random
import time
import torch
import tqdm
import requests
import string

import gensim.downloader as api
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage as sk
import skimage.io as skio
import torch.nn as nn
import torch.nn.functional as F
import tqdm.notebook

from collections import Counter, defaultdict
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from itertools import combinations
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.regularizers import l1_l2
from keras.models import Sequential
from keras.models import load_model
from keras.layers import (
    LSTM,
    Dense,
    Dropout,
    Bidirectional,
    GRU,
    Conv1D,
    MaxPooling1D,
    Flatten,
    LayerNormalization,
    SpatialDropout1D,
)
from keras.preprocessing.sequence import pad_sequences
from matplotlib.colors import hsv_to_rgb
from nltk import word_tokenize, tokenize
from numpy import array
from pickle import dump
from six import StringIO
from skimage import img_as_float, img_as_ubyte, img_as_float32, transform
from skimage.io import imread
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid
from tensorflow.keras.utils import to_categorical
from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple, Optional, Iterator
from IPython.display import Image

import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
PATH = '/content/drive/MyDrive/DMA Final Project/'
current_user = 'jessica_golden/'
PATH += current_user

assert torch.cuda.is_available()
cuda_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {cuda_device}")

Mounted at /content/drive


AssertionError: ignored

In [5]:
files = sorted(glob.glob(PATH+"*.json"))
files

['/content/drive/MyDrive/DMA Final Project/jessica_golden/StreamingHistory0.json',
 '/content/drive/MyDrive/DMA Final Project/jessica_golden/StreamingHistory1.json']

In [8]:
df0 = pd.read_json(files[0])
df0

,endTime,artistName,trackName,msPlayed
0,2022-10-03 16:03,Tori Templet,Butterfly Rain,167166
1,2022-10-04 18:40,Maisie Peters,The List,68976
2,2022-10-07 18:05,Catie Turner,Nothing,4335
3,2022-10-07 18:05,Billie Marten,Mice,1650
4,2022-10-07 18:06,Jacob Collier,In Too Deep (feat. Kiana Ledé),18280
...,...,...,...,...
9995,2023-05-14 07:38,Novo Amor,Decimal,178626
9996,2023-05-14 07:40,Olivia Dean,Ok Love You Bye,154386
9997,2023-05-14 07:45,Chelsea Cutler,Crazier Things (with Noah Kahan),264584
9998,2023-05-14 07:49,Zachary Knowles,blue bird,221076


In [9]:
df1 = pd.read_json(files[1])
df1

,endTime,artistName,trackName,msPlayed
0,2023-05-14 07:53,Jordan Hart,I Don't Want to Let You Go,215685
1,2023-05-14 07:56,Wild Rivers,Mayday,160083
2,2023-05-14 07:58,shy martin,can I call you back?,153058
3,2023-05-14 08:01,Ryan McMullan,Letting Go for a Little While,188795
4,2023-05-14 08:05,Harrison Storm,Breathe Again - Acoustic,198381
...,...,...,...,...
8203,2023-10-07 21:29,Anthony Amorim,Sometimes,234068
8204,2023-10-07 21:33,Juniper Vale,Fractions,230666
8205,2023-10-07 21:35,Simone,Manhattan,166493
8206,2023-10-07 21:36,Luz,we'll be fine,9204


In [12]:
# Combined df
df = pd.concat([df0, df1]).reset_index(drop=True)
df

,endTime,artistName,trackName,msPlayed
0,2022-10-03 16:03,Tori Templet,Butterfly Rain,167166
1,2022-10-04 18:40,Maisie Peters,The List,68976
2,2022-10-07 18:05,Catie Turner,Nothing,4335
3,2022-10-07 18:05,Billie Marten,Mice,1650
4,2022-10-07 18:06,Jacob Collier,In Too Deep (feat. Kiana Ledé),18280
...,...,...,...,...
18203,2023-10-07 21:29,Anthony Amorim,Sometimes,234068
18204,2023-10-07 21:33,Juniper Vale,Fractions,230666
18205,2023-10-07 21:35,Simone,Manhattan,166493
18206,2023-10-07 21:36,Luz,we'll be fine,9204


In [ ]:
user_spotify_history = [
    {'song': 'Shape of You', 'artist': 'Ed Sheeran', 'genre': 'Pop'},
    {'song': 'Believer', 'artist': 'Imagine Dragons', 'genre': 'Rock'},
]

def convert_song_to_vector(song_data):
    # TODO: use word2vec
    return np.random.rand(16)  # Random vector as a placeholder

song_vectors = [convert_song_to_vector(song) for song in user_spotify_history]

kmeans = KMeans(n_clusters=2)  # TODO: (auto-)tune the number of clusters
clusters = kmeans.fit_predict(song_vectors)

In [ ]:
song_vectors

[array([0.64437919, 0.38338009, 0.82427052, 0.07618439, 0.39084848,
        0.74179312, 0.63924124, 0.54701815, 0.79758016, 0.39034858,
        0.81681098, 0.4652238 , 0.74533003, 0.42781986, 0.85114868,
        0.07565683]),
 array([0.77754575, 0.75421325, 0.03563111, 0.90454029, 0.92273281,
        0.31774727, 0.92714807, 0.2995316 , 0.28225795, 0.27390212,
        0.0624514 , 0.70198776, 0.66250097, 0.92807152, 0.79441573,
        0.65959621])]

In [ ]:
plt.figure(figsize=(10, 6))
for curr_cluster_idx in range(5):
    curr_cluster_songs = song_vectors[clusters == curr_cluster_idx]
    plt.scatter(curr_cluster_songs[:, 0], curr_cluster_songs[:, 1], label=f'Cluster {curr_cluster_idx+1}')

plt.title('Song Clusters based on User Spotify History (Hypothetical)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()

TypeError: ignored

<Figure size 1000x600 with 0 Axes>

Teamwork: We all worked equally on this project!